[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fonnesbeck/instats_gp/blob/main/sessions/Session_2.ipynb)

# Session 2: Kernels, Likelihoods, and Model Building

## Learning Objectives

By the end of this session, you will:

- Understand different kernel families and their properties
- Learn kernel composition via addition and multiplication
- Understand marginal vs latent GP formulations
- Work with non-Gaussian likelihoods
- Build additive and multiplicative models for real-world data

This session builds directly on Session 1's foundations. We'll move beyond basic GP models to explore the rich toolkit of covariance functions and likelihood choices that make Gaussian processes so flexible for real-world problems.

In [ ]:
import os
import arviz as az
import numpy as np
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pymc as pm
import matplotlib.pyplot as plt

RNG = np.random.default_rng(RANDOM_SEED:= 8675309)

DATA_DIR = "../data/"
OUTPUT_DIR = "../output/"

print(f"PyMC version: {pm.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Polars version: {pl.__version__}")
print(f"ArviZ version: {az.__version__}")

In Session 1, we built intuition about Gaussian processes by connecting multivariate normal distributions to functions. We learned that the **covariance function** (or kernel) is the heart of a GP—it encodes our assumptions about how similar function values should be at different input locations. We also saw how to build simple GP models in PyMC using `pm.gp.Marginal` with Gaussian noise.

This session takes us deeper. Real-world problems rarely involve perfectly smooth functions with Gaussian noise. You might encounter:

- **Periodic patterns** that repeat over time (think seasonal sales, daily temperatures)
- **Long-term trends** which require longer length scales
- **Binary outcomes** where we need classification, not regression
- **Heavy-tailed noise** that makes outliers more likely than a normal distribution suggests

Fortunately, the GP framework is remarkably flexible. By choosing appropriate kernels and likelihoods, we can handle all these scenarios. Let's see how.

## Section 2.1: The Kernel Zoo

Kernels are the building blocks that define the structure of our GP. Different kernels encode different assumptions about function smoothness, periodicity, and behavior. Let's explore the most important kernel families and develop intuition for when to use each one.

In [ ]:
def plot_gp_prior_samples(X, cov_func, n_samples=5, title="GP Prior Samples"):
    """
    Plot samples from a GP prior to visualize kernel behavior.
    
    Parameters
    ----------
    X : array
        Input locations (1D array, will be reshaped to column vector)
    cov_func : PyMC covariance function
        The kernel to visualize
    n_samples : int
        Number of function samples to draw
    title : str
        Plot title
    """
    X = X.reshape(-1, 1)
    
    # Compute covariance matrix and add jitter for numerical stability
    K = cov_func(X).eval() + 1e-6 * np.eye(len(X))
    
    # Draw samples from the GP prior
    samples = RNG.multivariate_normal(np.zeros(len(X)), K, size=n_samples)
    
    # Create plotly figure
    fig = go.Figure()
    
    for i in range(n_samples):
        fig.add_trace(go.Scatter(
            x=X.flatten(),
            y=samples[i],
            mode='lines',
            name=f'Sample {i+1}',
            opacity=0.7,
            line=dict(width=2)
        ))
    
    fig.update_layout(
        title=title,
        xaxis_title='X',
        yaxis_title='f(x)',
        hovermode='x unified',
        width=900,
        height=500
    )
    
    return fig

### Matérn Family

The Matérn kernels generalize the ExpQuad by controlling smoothness through a parameter $\nu$. Smaller $\nu$ means less smooth. PyMC provides three variants:

- **Matérn 1/2**: Continuous but not differentiable (roughest)
- **Matérn 3/2**: Once differentiable (medium smoothness)
- **Matérn 5/2**: Twice differentiable (smooth, but not as extreme as ExpQuad)

Use Matérn when you want control over smoothness or when your function might not be infinitely smooth.

In [ ]:
# Create input points
X_grid = np.linspace(0, 10, 200)

# Fixed parameters for fair comparison
lengthscale = 1.0
amplitude = 1.0

# Define three kernels with different smoothness properties
kernels = {
    'ExpQuad (RBF)': amplitude**2 * pm.gp.cov.ExpQuad(1, lengthscale),
    'Matern52': amplitude**2 * pm.gp.cov.Matern52(1, lengthscale),
    'Matern32': amplitude**2 * pm.gp.cov.Matern32(1, lengthscale),
}

# Plot prior samples for each kernel
for name, cov in kernels.items():
    fig = plot_gp_prior_samples(X_grid, cov, n_samples=5, title=f'{name} Kernel - Prior Samples')
    fig.show()

Notice how the functions become progressively smoother as we go from Matérn 1/2 to 5/2. The Matérn 1/2 (also called Exponential kernel) produces jagged, continuous but non-differentiable functions—perfect for modeling rough phenomena.

### The Rational Quadratic Kernel: Modeling Multiple Scales

So far, we've seen kernels with a single lengthscale parameter that controls how quickly correlations decay with distance. But what if your data has variation at multiple timescales simultaneously? Imagine a phenomenon with both short-term fluctuations and long-term trends that can't be cleanly decomposed into additive components.

The **Rational Quadratic (RQ) kernel** addresses this by acting as a **scale mixture**—essentially combining many squared exponential kernels with different lengthscales. Mathematically:

$$
k_{RQ}(x, x') = \sigma^2 \left(1 + \frac{(x - x')^2}{2\alpha\ell^2}\right)^{-\alpha}
$$

The key parameter is $\alpha$ (alpha), which controls the mixture:
- **Small α** (e.g., 0.5-2): Heavy-tailed behavior, more variation across scales
- **Large α** (e.g., 100+): Approaches the squared exponential kernel
- **α → ∞**: Becomes exactly the ExpQuad kernel

Think of α as controlling how "multi-scale" your function is. When you're not sure whether your phenomenon has one dominant lengthscale or many, the RQ kernel lets the data decide.

Let's visualize how α affects the prior samples to build intuition.

In [ ]:
X_grid = np.linspace(0, 10, 200)

lengthscale = 1.0
amplitude = 1.0

alpha_values = [0.5, 2.0, 10.0, 100.0]

for alpha in alpha_values:
    cov = amplitude**2 * pm.gp.cov.RatQuad(1, ls=lengthscale, alpha=alpha)
    fig = plot_gp_prior_samples(
        X_grid,
        cov,
        n_samples=5,
        title=f'Rational Quadratic Kernel (α = {alpha})'
    )
    fig.show()

### Understanding the Rational Quadratic Through Its Samples

Looking at these prior samples across different α values reveals important patterns:

**With α = 0.5**: The functions are quite rough and can change behavior dramatically over short distances. This heavy-tailed behavior allows for surprising local variations while still maintaining some global structure. You'd use this when you expect your data to have unpredictable multi-scale variation.

**With α = 2.0**: We see moderate smoothness with the ability to capture both rapid local changes and slower global trends. This is often a good middle ground when you're unsure about the scale structure of your data.

**With α = 10.0**: The functions become noticeably smoother and more predictable, approaching the behavior of our Matérn kernels. The multi-scale mixing is still present but less pronounced.

**With α = 100.0**: These functions are nearly identical to what we'd get from an ExpQuad kernel—silky smooth and infinitely differentiable. At this point, there's effectively only one dominant lengthscale.

**Practical guidance**: Start with Matérn 5/2 for most problems. If you find yourself torn between multiple lengthscale values (the data seems to have both fine details and broad trends), try RQ with α ~ 2-5. If posterior estimates push α very high (> 50), you might as well use ExpQuad. If α stays very low (< 1), your data truly has multi-scale structure that can't be captured by a single lengthscale.

**Important caveat**: The RQ kernel can be more challenging for inference because it has one more hyperparameter than ExpQuad or Matérn. Use it when the extra flexibility is genuinely needed, not as a default.

### Periodic Kernels for Seasonal Patterns

Many real-world phenomena repeat: daily temperature cycles, weekly sales patterns, annual seasonal effects. For these, we need a kernel that encodes periodicity. The Periodic kernel is perfect for this.

The Periodic kernel has two key parameters beyond amplitude:
- **period**: The length of one complete cycle
- **lengthscale**: Controls smoothness *within* each period (how similar nearby points within a cycle should be)

Let's visualize periodic patterns and see how they differ from the stationary kernels we've explored.

In [ ]:
# Demonstrate periodic kernel with annual period
X_grid = np.linspace(0, 20, 400)  # Extended range to see multiple periods

period = 5.0  # Complete cycle every 5 units
lengthscale = 1.0
amplitude = 1.0

cov_periodic = amplitude**2 * pm.gp.cov.Periodic(1, period=period, ls=lengthscale)

fig = plot_gp_prior_samples(
    X_grid, 
    cov_periodic, 
    n_samples=5,
    title=f'Periodic Kernel (period={period}, lengthscale={lengthscale})'
)
fig.show()

### Interpreting Periodic Patterns

Notice how the functions repeat with the specified period. The pattern from $x=0$ to $x=5$ is similar to the pattern from $x=5$ to $x=10$, and so on. This is exactly what we want for seasonal data.

The lengthscale parameter controls smoothness *within* each period. A smaller lengthscale would allow more wiggly behavior within each cycle, while a larger lengthscale would make each cycle smoother.

**Real-world example**: If you're modeling daily data with weekly patterns, you'd set `period=7`. The GP would then learn that Mondays tend to be similar to other Mondays, Tuesdays to Tuesdays, and so forth, while still being flexible about the exact pattern.

### The Linear Kernel for Trends

Sometimes your data contains linear or polynomial trends. The Linear kernel captures this by computing covariance proportional to the inner product of input locations. Unlike the stationary kernels we've seen, the Linear kernel is *non-stationary*—the covariance depends on the absolute position, not just the distance between points.

In [ ]:
# Demonstrate linear kernel
X_grid = np.linspace(0, 10, 200)

c = 1.0  # Center point
variance = 1.0

cov_linear = variance * pm.gp.cov.Linear(1, c=c)

fig = plot_gp_prior_samples(
    X_grid,
    cov_linear,
    n_samples=5,
    title='Linear Kernel - Prior Samples'
)
fig.show()

### Understanding the Linear Kernel

These functions are predominantly linear, though you'll notice they're not perfectly straight lines—there's some flexibility around the linear trend. The Linear kernel alone is rarely sufficient for real data, but it becomes powerful when *combined* with other kernels (which we'll explore in Section 2.4).

The parameter `c` acts as a center point or offset. The covariance between two points $x$ and $x'$ is proportional to $(x - c)(x' - c)$. This makes the variance grow as you move away from $c$.

### Choosing the Right Kernel: A Practical Guide

Now that we've explored the kernel zoo, you might be wondering: "How do I choose the right kernel for my data?" The choice of kernel determines almost all the generalization properties of your GP model, so this decision matters immensely.

Here's a practical decision framework based on what you know (or suspect) about your data:

#### Start With These Questions

**Is your function smooth everywhere, or does it have sharp features?**
- Smooth everywhere → ExpQuad or Matérn 5/2
- Occasional sharp features or kinks → Matérn 3/2
- Very rough or irregular → Matérn 1/2 (rarely used)

**Does your data have obvious periodic patterns?**
- Clear, exact repetition → Periodic kernel (or composed with others)
- Periodic but changing over time → Locally Periodic (ExpQuad × Periodic)
- No periodicity → Skip Periodic entirely

**Are there multiple timescales or characteristic lengths?**
- Single dominant scale → Matérn 5/2 or ExpQuad
- Multiple scales that can be separated → Additive composition (k₁ + k₂)
- Multiple scales that interact → Rational Quadratic or multiplicative composition

**Is there a trend component?**
- Linear trend → Include Linear kernel (usually composed with others)
- Polynomial trend → Linear × Linear for quadratic, etc.
- Complex trend → Use smooth kernel with long lengthscale

#### The Conservative Default

**When in doubt, start with Matérn 5/2.** It's smooth enough for most applications without being overly smooth like ExpQuad, and it generalizes well across problem domains. This has become the de facto recommendation in modern GP practice.

#### Warning Signs of Poor Kernel Choice

Watch for these indicators that your kernel might be misspecified:

**Shrinking lengthscale with more data**: If your lengthscale posterior keeps getting smaller as you add more observations, your kernel is likely too smooth for the data. The model is trying to capture roughness by shrinking the lengthscale indefinitely. Consider switching to a rougher kernel (e.g., Matérn 5/2 → Matérn 3/2) or adding complexity through composition.

**Very wide posterior distributions**: If hyperparameter posteriors are extremely broad even with substantial data, the kernel might not match the data structure. Try a different kernel family or consider composition.

**Systematic residual patterns**: If your posterior predictive checks reveal clear patterns in the residuals (e.g., unmodeled periodicity, unmodeled trend), your kernel is missing structure. Add appropriate components through composition.

Let's move on to see these kernels in action through model building.

### A Workflow for Kernel Selection

Rather than agonizing over the perfect kernel upfront, follow this iterative workflow:

**Step 1: Start Simple**

Begin with a single Matérn 5/2 kernel with weakly informative priors. This gives you a baseline and helps you understand the data's basic structure.

**Step 2: Examine Residuals**

After fitting, plot the difference between your observations and posterior mean predictions. Look for patterns:
- Regular oscillations → Consider adding Periodic
- Long-term drift → Consider adding Linear or smooth kernel with long lengthscale
- Different behavior in different regions → Consider multiplicative compositions or change-point models
- Outliers → Consider robust likelihoods (Student-T)

**Step 3: Add Complexity Incrementally**

Based on residual patterns, add kernel components one at a time. Refit after each addition and verify the new component improves the model.

This workflow prevents both underfitting (too simple) and overfitting (too complex) while maintaining computational efficiency. Now let's see how to actually build and fit these models in PyMC.

## Section 2.2: Marginal Likelihood with pm.gp.Marginal

When your data are continuous observations with Gaussian noise, `pm.gp.Marginal` provides the most efficient implementation. It analytically integrates out the latent GP function, directly computing the marginal likelihood $p(y | x)$ without sampling the intermediate function values.

### Mathematical Intuition

Recall that a GP models a function $f(x)$, and our observations are:

$$
\begin{align}
f(x) &\sim \mathcal{GP}(m(x), k(x, x')) \\
y &= f(x) + \epsilon \\
\epsilon &\sim \mathcal{N}(0, \sigma^2)
\end{align}
$$

Because both the GP prior and the Gaussian noise are normal distributions, we can analytically integrate out $f$ to get:

$$
y \sim \mathcal{N}(m(X), K(X, X) + \sigma^2 I)
$$

This marginalization is exact and efficient—we don't need to sample the potentially high-dimensional latent function. The `marginal_likelihood` method implements this approach.

### When to Use pm.gp.Marginal

Choose `pm.gp.Marginal` when:
- Your likelihood is Gaussian (normal)
- You have continuous-valued observations
- You want the fastest inference possible
- You don't need samples from the latent function itself

Let's work through a complete example using real data.

### Example: Recovering a Known Latent Function

We'll start with a controlled example where we know the true underlying function. This lets us see exactly how well the GP recovers the latent function from noisy observations—a perfect way to build intuition before tackling real-world data with multiple patterns.

We'll generate synthetic data by:
1. Sampling a single function from a Matérn 5/2 GP prior (our "ground truth")
2. Adding Gaussian noise to create observations
3. Using `pm.gp.Marginal` to recover the latent function from the noisy data

This approach removes domain complexity and lets us focus purely on GP mechanics: how the model learns lengthscales, amplitudes, and noise levels from data.

In [ ]:
# Generate simulated data from a known GP
np.random.seed(RANDOM_SEED)  # For reproducibility

n = 100  # Number of data points
X = np.linspace(0, 10, n)[:, None]  # Inputs arranged as column vector

# Define the true covariance function and parameters
ell_true = 1.0
eta_true = 3.0
cov_func_true = eta_true**2 * pm.gp.cov.Matern52(1, ell_true)

# Zero mean function
mean_func = pm.gp.mean.Zero()

# Draw one sample from the GP prior - this is our "true" latent function
K_true = cov_func_true(X).eval() + 1e-8 * np.eye(n)  # Add jitter for numerical stability
f_true = RNG.multivariate_normal(mean_func(X).eval(), K_true)

# Add Gaussian noise to create observations
sigma_true = 2.0
y = f_true + sigma_true * RNG.standard_normal(n)

# Create polars dataframe for plotting
df_sim = pl.DataFrame({
    'x': X.flatten(),
    'y': y,
    'f_true': f_true
})

# Visualize the data
fig = go.Figure()

# True latent function
fig.add_trace(go.Scatter(
    x=df_sim['x'],
    y=df_sim['f_true'],
    mode='lines',
    name='True latent function f(x)',
    line=dict(color='dodgerblue', width=3)
))

# Noisy observations
fig.add_trace(go.Scatter(
    x=df_sim['x'],
    y=df_sim['y'],
    mode='markers',
    name='Observed data y = f(x) + noise',
    marker=dict(size=5, color='black', opacity=0.6)
))

fig.update_layout(
    title='Simulated GP Data: Known Ground Truth',
    xaxis_title='X',
    yaxis_title='Y',
    hovermode='x unified',
    width=900,
    height=500
)

fig.show()

### Observing the Data Structure

The blue line shows our true latent function $f(x)$—a single draw from a Matérn 5/2 GP prior. Notice its smooth, gradually varying behavior with occasional changes in direction. This is characteristic of the Matérn 5/2 kernel: twice differentiable but not infinitely smooth.

The black points show our noisy observations $y = f(x) + \epsilon$, where $\epsilon \sim \mathcal{N}(0, 2^2)$. The noise obscures the true function, but there's clearly structure in the data—points nearby in $x$ tend to have similar $y$ values.

Our task: recover the blue line (or something close to it) from only the black points. Can a GP learn the appropriate lengthscale, amplitude, and noise level to reconstruct the latent function? Let's find out.

### Building the Model

We'll specify priors for the hyperparameters (lengthscale, amplitude, and noise level), then use `pm.gp.Marginal` to efficiently compute the marginal likelihood. Since we generated this data, we know the true parameter values:

- True lengthscale: $\ell = 1.0$
- True amplitude: $\eta = 3.0$
- True noise: $\sigma = 2.0$

We'll use weakly informative priors that don't reveal these values—the model must learn them from data alone.

In [ ]:
with pm.Model() as marginal_model:
    # Priors for hyperparameters
    ell = pm.Gamma('ell', alpha=2, beta=1)
    eta = pm.HalfCauchy('eta', beta=5)
    
    # Define covariance function
    cov_func = eta**2 * pm.gp.cov.Matern52(1, ell)
    
    # Create the GP (mean function defaults to zero)
    gp = pm.gp.Marginal(cov_func=cov_func)
    
    # Noise level
    sigma = pm.HalfCauchy('sigma', beta=5)
    
    # Marginal likelihood
    y_obs = gp.marginal_likelihood('y', X=X, y=y, sigma=sigma)
    
    # Sample from posterior
    marginal_trace = pm.sample(
        500,
        tune=1000,
        nuts_sampler='nutpie',
        chains=2,
        random_seed=RNG
    )

### Understanding the Model Components

Let's break down what we specified:

**Lengthscale prior (`ell`)**: Gamma(2, 1) has most mass between 0.5 and 5. This weakly suggests moderate correlation distances without being too informative. The true value (1.0) falls comfortably within this range.

**Amplitude prior (`eta`)**: HalfCauchy(5) has heavy tails, allowing for a wide range of vertical scales. It's weakly informative—it won't force a particular amplitude but prevents unreasonably extreme values.

**Noise prior (`sigma`)**: Another HalfCauchy(5), placing weak prior mass on moderate noise levels. The data will strongly inform this parameter since we have 100 observations.

The key method call is `gp.marginal_likelihood`, which analytically integrates out the latent function $f$ to directly compute $p(y | \theta)$. This is much faster than sampling $f$ explicitly (which we'd need with `pm.gp.Latent`).

Let's check if inference succeeded.

### Checking Inference Diagnostics

Before trusting our results, let's verify that sampling worked well. We'll check R-hat values and examine trace plots.

In [ ]:
# Check R-hat values and ESS
summary = az.summary(marginal_trace, var_names=['ell', 'eta', 'sigma'])
print(summary)
print(f"\nTrue parameter values:")
print(f"  ell_true  = {ell_true}")
print(f"  eta_true  = {eta_true}")
print(f"  sigma_true = {sigma_true}")

# Check for divergences
divergences = marginal_trace.sample_stats['diverging'].sum().item()
print(f"\nNumber of divergences: {divergences}")

### Interpreting the Diagnostics

Excellent! Our R-hat values are all close to 1.0, indicating convergence. The effective sample sizes (ESS) are in the hundreds, giving us reliable posterior estimates. Most importantly, zero divergences means NUTS had no trouble exploring the posterior.

Looking at the posterior means, notice how close they are to the true values:
- The posterior mean for `ell` should be near 1.0
- The posterior mean for `eta` should be near 3.0
- The posterior mean for `sigma` should be near 2.0

The GP has successfully learned the hyperparameters from data alone! The posterior distributions capture our uncertainty—we're not perfectly certain about the true values (especially with only 100 data points), but the data provide strong evidence about the characteristic scales.

Let's visualize the posterior to see this more clearly.

In [ ]:
# Visualize posterior distributions
az.plot_trace(marginal_trace, var_names=['ell', 'eta', 'sigma'], backend_kwargs={'constrained_layout': True});

### Making Predictions

Now comes the exciting part: using our trained GP to make predictions at new input locations. We'll use the `conditional` method to compute the posterior predictive distribution.

In [ ]:
# New X locations for prediction (including extrapolation)
X_new = np.linspace(-1, 12, 300)[:, None]

with marginal_model:
    # Conditional distribution for the latent function
    f_pred = gp.conditional('f_pred', X_new)
    
    # Sample from the posterior predictive
    marginal_ppc = pm.sample_posterior_predictive(
        marginal_trace,
        var_names=['f_pred'],
        random_seed=RNG
    )

### Visualizing Posterior Predictions

Now comes the exciting part: can the GP recover the true latent function from noisy observations? Let's plot the posterior mean along with credible intervals to show our uncertainty. We'll also overlay the true function to see how close we got.

In [ ]:
def plot_gp_posterior(ppc, X_pred, X, y, var_name='f_pred',
                      title='GP Posterior Predictions', xaxis_title='X', yaxis_title='Y',
                      show_training_range=True):

    f_pred_samples = ppc.posterior_predictive[var_name].values

    f_pred_mean = f_pred_samples.mean(axis=(0, 1))
    f_pred_lower = np.percentile(f_pred_samples, 2.5, axis=(0, 1))
    f_pred_upper = np.percentile(f_pred_samples, 97.5, axis=(0, 1))

    # Calculate y-axis range from data
    y_min = y.min()
    y_max = y.max()
    y_range = y_max - y_min
    y_padding = 0.1 * y_range  # 10% padding

    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=X_pred.flatten(),
        y=f_pred_upper,
        mode='lines',
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    )).add_trace(go.Scatter(
        x=X_pred.flatten(),
        y=f_pred_lower,
        mode='lines',
        fill='tonexty',
        fillcolor='rgba(255, 0, 0, 0.2)',
        line=dict(width=0),
        name='95% Credible Interval'
    )).add_trace(go.Scatter(
        x=X_pred.flatten(),
        y=f_pred_mean,
        mode='lines',
        name='Posterior mean',
        line=dict(color='red', width=2)
    )).add_trace(go.Scatter(
        x=X.flatten(),
        y=y,
        mode='markers',
        name='Observed data',
        marker=dict(size=4, color='black')
    )).update_layout(
        title=title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        yaxis=dict(range=[y_min - y_padding, y_max + y_padding]),
        hovermode='x unified',
        width=900,
        height=500,
    )
    
    return fig

plot_gp_posterior(
    marginal_ppc,
    X_new,
    X,
    y,
    title='GP Marginal: Posterior Predictions with Uncertainty',
    xaxis_title='Years since 1990',
    yaxis_title='CO₂ (ppm)'
)

### Interpreting the Predictions

Remarkable! The GP has successfully recovered the true latent function (blue line) from noisy observations:

**Within the training region** ($0 < x < 10$): The posterior mean (red line) tracks the true function closely. The 95% credible interval (shaded) captures the true function almost everywhere, reflecting appropriate uncertainty. Where observations are dense and informative, the credible interval is narrow. This is Bayesian inference at work—the data strongly constrain what the latent function could be.

**In extrapolation regions** ($x < 0$ or $x > 10$): Uncertainty grows rapidly. Without data to anchor predictions, the GP reverts to its prior belief: functions should stay near zero (the mean function) but could wander. Notice how the credible interval widens—this is the model honestly admitting "I don't know what happens out here."

**Key insight**: The GP didn't just fit a curve through the points—it learned the underlying **process** that generated the data. It inferred the lengthscale (how quickly correlations decay), the amplitude (vertical scale of variation), and the noise level (observation uncertainty). With these learned parameters, it reconstructed the latent function.

This is the power of `pm.gp.Marginal` for Gaussian likelihood problems: fast, exact inference that properly quantifies uncertainty.

## 🤖 EXERCISE: Explore Different Kernels

Now that you've seen a successful GP fit, try experimenting with different kernel choices!

### Your Task

Modify the model above to use different kernels and observe how they affect the fit. Try:

1. **ExpQuad kernel**: Replace `Matern52` with `ExpQuad`—does it over-smooth or under-smooth?
2. **Matérn 3/2 kernel**: Replace with `Matern32`—does it capture sharper features?
3. **Misspecified prior**: Change the lengthscale prior to something that is too tight around an inappropriate value—does it still recover the truth?

Use your LLM to help you make these modifications and interpret the results.

In [ ]:
# Modify the kernel and re-fit

with pm.Model() as exercise_model:
    ell = pm.Gamma('ell', alpha=2, beta=1)
    eta = pm.HalfCauchy('eta', beta=5)
    
    # MODIFY THIS LINE - try different kernels
    cov_func = eta**2 * pm.gp.cov.Matern52(1, ell)
    
    gp = pm.gp.Marginal(cov_func=cov_func)
    sigma = pm.HalfCauchy('sigma', beta=5)
    y_obs = gp.marginal_likelihood('y', X=X, y=y, sigma=sigma)
    
    exercise_trace = pm.sample(
        500,
        tune=1000,
        nuts_sampler='nutpie',
        chains=2,
        random_seed=RNG
    )

# Visualize results
az.plot_trace(exercise_trace, var_names=['ell', 'eta', 'sigma'], backend_kwargs={'constrained_layout': True});

Let's plot the posterior predictions to see how your kernel choice affected the fit!

In [ ]:
with exercise_model:
    f_pred = gp.conditional('f_pred', X_new)
    exercise_ppc = pm.sample_posterior_predictive(
        exercise_trace,
        var_names=['f_pred'],
        random_seed=RNG
    )

# Plot using the same function
plot_gp_posterior(
    exercise_ppc,
    X_new,
    X,
    y,
    title='GP with Alternative Kernel',
    xaxis_title='X',
    yaxis_title='Y'
)

## Section 2.3: Latent GPs with pm.gp.Latent

While `pm.gp.Marginal` is efficient for Gaussian likelihoods, many real-world problems don't produce continuous measurements with Gaussian noise. Consider:

- **Classification**: Predicting whether a patient has a disease (binary outcome)
- **Count data**: Number of events in a time period (discrete, non-negative)
- **Survival analysis**: Time until an event occurs (censored data)

For these cases, we need `pm.gp.Latent`, which keeps the GP function as a latent variable in the model and allows arbitrary likelihood functions.

### The Latent GP Approach

Instead of marginalizing out the GP, we explicitly sample it:

$$
\begin{align}
f &\sim \mathcal{GP}(m, k) \\
y_i &\sim \text{Likelihood}(g(f(x_i)), \theta)
\end{align}
$$

where $g$ is a link function (like logistic for binary outcomes) and Likelihood can be any distribution (Bernoulli, Poisson, Student-T, etc.).

### Trade-offs: Latent vs Marginal

**pm.gp.Marginal**:
- ✓ Fast: analytically integrates out the GP
- ✓ Efficient: fewer parameters to sample
- ✗ Limited: only works with Gaussian likelihoods

**pm.gp.Latent**:
- ✓ Flexible: works with any likelihood
- ✓ Includes GP samples in posterior
- ✗ Slower: must sample high-dimensional latent function
- ✗ Can have sampling challenges

Let's work through a classification example to see `pm.gp.Latent` in action.

### Example: Poisson Regression for Count Data

Many real-world applications involve count data—discrete, non-negative outcomes like the number of events occurring in a time period. The **Poisson distribution** is the natural choice for modeling counts, and when we expect the rate to vary smoothly over time, a GP with Poisson likelihood becomes a powerful tool.

We'll work with the famous British coal mining disasters dataset, which records 191 mining disasters from 1851 to 1962. Our goal: model how the disaster rate changes over this period using a GP to capture smooth temporal trends without imposing a rigid parametric form.

The Poisson likelihood requires a positive rate parameter $\lambda$, so we'll use a **log link function**: the GP models $\log(\lambda)$, ensuring $\lambda > 0$. Let's load the data and bin it into annual counts.

In [ ]:
# Load coal mining disasters data
coal_data_path = pm.get_data('coal.csv')
coal_df_raw = pl.read_csv(coal_data_path, has_header=False, new_columns=['year'])

print(f"Total disasters recorded: {len(coal_df_raw)}")
print(f"Time period: {coal_df_raw['year'].min():.1f} to {coal_df_raw['year'].max():.1f}")

# Bin into annual counts
# Create bins for each year from 1851 to 1963 (to capture all disasters)
year_min = int(np.floor(coal_df_raw['year'].min()))
year_max = int(np.ceil(coal_df_raw['year'].max()))

# Count disasters per year
year_counts = []
years = []
for year in range(year_min, year_max + 1):
    count = coal_df_raw.filter(
        (pl.col('year') >= year) & (pl.col('year') < year + 1)
    ).height
    year_counts.append(count)
    years.append(year)

# Create DataFrame with annual counts
df_coal = pl.DataFrame({
    'year': years,
    'count': year_counts,
    'year_scaled': [(y - year_min) for y in years]  # Years since 1851
})

print(f"\nAnnual counts range: {min(year_counts)} to {max(year_counts)}")
print(f"\nFirst few years:")
print(df_coal.head(10))

# Visualize the annual disaster counts
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_coal['year_scaled'],
    y=df_coal['count'],
    mode='markers',
    name='Annual disaster count',
    marker=dict(size=6, color='black', opacity=0.7)
))

fig.update_layout(
    title='British Coal Mining Disasters (1851-1962)',
    xaxis_title='Years since 1851',
    yaxis_title='Number of disasters per year',
    hovermode='x unified',
    width=900,
    height=500
)

fig.show()

### Understanding the Coal Mining Data

Looking at the plot, we see clear temporal patterns in the annual disaster counts:

**Early period (1851-1890s)**: Disaster rates are notably higher, with several years experiencing 5-6 disasters. This reflects the dangerous conditions of 19th-century mining—inadequate safety regulations, primitive equipment, and poor ventilation.

**Later period (1900s onward)**: The disaster rate appears to decline, with most years having 0-2 disasters. This likely reflects improving safety standards, better mining technology, and stricter regulations following major disasters.

**Key observation**: The rate appears to change smoothly over time rather than in discrete jumps. This makes the coal mining dataset perfect for GP modeling—we want to capture this smooth temporal trend without assuming a specific parametric form (like exponential decay).

Notice these are **count data**—non-negative integers, not continuous values. A Gaussian likelihood would be inappropriate since it could predict negative disaster counts. The Poisson distribution is designed exactly for this type of data.

### Building the Poisson GP Model

Now we'll use `pm.gp.Latent` with a Poisson likelihood to model the disaster rate. The key modeling choices:

**Link function**: The Poisson distribution parameterizes the rate $\lambda > 0$, but our GP can range over all real numbers. We use the **log link**: $\log(\lambda) = f(t)$, which means $\lambda = \exp(f(t))$. This ensures the rate is always positive regardless of what the GP predicts.

**Kernel choice**: We'll use Matérn 5/2, which allows smooth but not infinitely smooth variations—appropriate for a disaster rate that might change gradually as mining practices evolve. A lengthscale prior that's weakly informative over the ~110-year timespan makes sense.

**Prior considerations**: We need to be thoughtful about the GP prior on the log-scale. Since disasters are relatively rare events (at most 6 per year), $\log(\lambda)$ should be modest—probably between -2 (≈0.14 disasters/year) and 2 (≈7.4 disasters/year).

In [ ]:
# Prepare data for modeling
X_coal = df_coal['year_scaled'].to_numpy().reshape(-1, 1)
y_coal = df_coal['count'].to_numpy()

with pm.Model() as coal_model:
    # Hyperparameter priors
    # Lengthscale: expect variation on the scale of 10-30 years
    ell = pm.Gamma('ell', alpha=3, beta=0.2)
    
    # Amplitude: controls variation in log-rate
    eta = pm.HalfNormal('eta', sigma=2)
    
    # Covariance function - Matérn 5/2 for smooth but flexible trends
    cov_func = eta**2 * pm.gp.cov.Matern52(1, ell)
    
    # Latent GP
    gp = pm.gp.Latent(cov_func=cov_func)
    
    # GP prior - this is the log-rate
    f = gp.prior('f', X=X_coal)
    
    # Transform to rate via exponential (inverse of log link)
    lam = pm.Deterministic('lambda', pm.math.exp(f))
    
    # Poisson likelihood
    y_obs = pm.Poisson('y', mu=lam, observed=y_coal)
    
    # Sample from posterior
    coal_trace = pm.sample(
        500,
        tune=500,
        nuts_sampler='nutpie',
        chains=2,
        target_accept=0.9,
        random_seed=RNG
    )

### Understanding the Poisson GP Structure

The critical components of this model differ from our earlier examples:

**`gp.prior('f', X=X_coal)`**: This explicitly samples the latent GP as a variable in our model. The vector `f` represents $\log(\lambda)$ at each year—the log-rate of disasters.

**`pm.Deterministic('lambda', pm.math.exp(f))`**: This is the **inverse link function**, transforming from log-rate to actual rate. By tracking it as a Deterministic variable, we can examine the disaster rate $\lambda$ directly in our posterior, not just the log-rate. This makes interpretation much easier.

**`pm.Poisson('y', mu=lam, observed=y_coal)`**: The Poisson likelihood connects our latent rate to the observed counts. This is fundamentally different from Gaussian or Bernoulli—the Poisson naturally handles non-negative integer counts and has variance equal to its mean.

**Why we can't use `pm.gp.Marginal`**: There's no way to analytically marginalize a Poisson likelihood. We must explicitly sample the latent function `f`, which is what `pm.gp.Latent` does. This makes inference slower but enables the flexibility to use non-Gaussian likelihoods.

Let's check how well sampling worked.

In [ ]:
# Check convergence
summary_coal = az.summary(coal_trace, var_names=['ell', 'eta'])
print(summary_coal)

# Check divergences
divergences_coal = coal_trace.sample_stats['diverging'].sum().item()
print(f"\nNumber of divergences: {divergences_coal}")

if divergences_coal == 0:
    print("✓ No divergences - sampling worked well!")
else:
    print(f"⚠ Warning: {divergences_coal} divergences detected. Consider reparameterization.")

### Visualizing the Posterior Disaster Rate

Now let's see how well our GP captured the temporal trend in disaster rates. We'll plot the posterior mean rate $\lambda(t)$ along with credible intervals, overlaying the observed annual counts.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

pm.gp.util.plot_gp_dist(
    ax=ax,
    x=df_coal['year_scaled'].to_numpy(), 
    samples=coal_trace.posterior['lambda'].values.reshape(-1, len(df_coal)),
    plot_samples=False, 
    palette='Reds'
)

ax.plot(
    df_coal['year_scaled'].to_numpy(),
    df_coal['count'].to_numpy(),
    'ko',
    markersize=4,
    alpha=0.6,
    label='Observed annual counts'
)

ax.set_xlabel('Years since 1851')
ax.set_ylabel('Disaster rate (disasters/year)')
ax.set_title('GP Poisson Regression: Coal Mining Disaster Rate Over Time')
ax.legend();

### Interpreting the Poisson GP Results

This is a beautiful demonstration of GP flexibility with non-Gaussian likelihoods. The posterior mean rate (red line) captures the historical trend in coal mining disasters:

**Early period (1851-1890s)**: The estimated disaster rate is high, around 3-4 disasters per year. The model has learned that Victorian-era mining was substantially more dangerous, reflected in the elevated rate during this period.

**Transition period (~1890-1900)**: The rate begins to decline as safety practices gradually improved. The GP smoothly captures this transition without us imposing any particular functional form—no exponential decay, no linear trend, just letting the data speak.

**Later period (1900-1962)**: The disaster rate settles to a lower level, around 1-2 disasters per year. While still tragic, this represents a substantial improvement from the earlier period.

**Uncertainty quantification**: The credible interval (shaded region) reflects our uncertainty about the true underlying rate. Notice it's wider in periods with high variability in the observed counts—the Poisson variance equals its mean, so high-rate periods naturally have more scatter.

**The power of the log link**: By modeling $\log(\lambda)$ with a GP and exponentiating, we guaranteed $\lambda > 0$ throughout. This is essential for count data—we never predict negative disaster rates, even though our GP can range over all real numbers.

Now let's move on to one of the most powerful features of GPs: kernel composition.

## 🤖 EXERCISE: Fitting a GP to Atmospheric CO₂ Data

Now it's your turn to fit a GP model to real-world data. We'll use the famous Mauna Loa CO₂ dataset—the "Keeling Curve"—which has become one of the most important scientific time series in climate science.

### Understanding the Mauna Loa CO₂ Dataset

Since 1958, researchers at the Mauna Loa Observatory in Hawaii have continuously measured atmospheric CO₂ concentrations. This remote location in the middle of the Pacific Ocean provides measurements relatively free from local pollution sources, making it an ideal monitoring station for global atmospheric composition.

The Keeling Curve exhibits two striking patterns that you'll notice when you plot the data:

1. **Long-term increasing trend** (decades-scale) due to cumulative anthropogenic emissions from fossil fuels, deforestation, and industrial activity
2. **Annual seasonal cycles** (yearly oscillations) due to Northern Hemisphere vegetation dynamics—plants absorb CO₂ during spring/summer growth and release it during fall/winter decay

For this exercise, we'll focus on a 10-year window (1990-2000) to keep the computation manageable.

In [ ]:
# Load Mauna Loa CO2 data from PyMC datasets
co2_file = pm.get_data("monthly_in_situ_co2_mlo.csv")

df_raw = pl.read_csv(
    co2_file,
    skip_rows=57,  # Skip metadata header
    has_header=False,
    null_values=["-99.99"],  # Handle missing value indicator
    schema_overrides={
        "column_1": pl.Int64,    # year
        "column_2": pl.Int64,    # month
        "column_3": pl.Float64,  # decimal_date
        "column_4": pl.Float64,  # monthly_avg
        "column_5": pl.Float64,  # CO2
        "column_6": pl.Float64,  # seasonally_adjusted
        "column_7": pl.Float64,  # fit
        "column_8": pl.Float64,  # seasonally_adjusted_fit
        "column_9": pl.Float64,  # CO2_filled
        "column_10": pl.Float64, # seasonally_adjusted_filled
    }
)

df_raw.columns = [
    "year", "month", "decimal_date", "monthly_avg", "CO2", "seasonally_adjusted",
    "fit", "seasonally_adjusted_fit", "CO2_filled", "seasonally_adjusted_filled"
]

df_co2 = (
    df_raw
    .filter((pl.col("year") >= 1990) & (pl.col("year") < 2000))
    .with_columns(
        (pl.col("year") + (pl.col("month") - 1) / 12 - 1990).alias("time")
    )
    .select(["time", "CO2"])
    .drop_nulls()
)

X_co2 = df_co2.select("time").to_numpy().reshape(-1, 1)
y_co2 = df_co2.select("CO2").to_numpy().flatten()

print(f"Data shape: {X_co2.shape[0]} monthly observations from 1990-2000")
print(f"CO2 range: {y_co2.min():.2f} to {y_co2.max():.2f} ppm")

go.Figure().add_trace(go.Scatter(
    x=df_co2["time"].to_numpy(),
    y=df_co2["CO2"].to_numpy(),
    mode="markers",
    marker=dict(size=4, color="steelblue"),
    name="Observed CO₂"
)).update_layout(
    title="Atmospheric CO₂ at Mauna Loa (1990-2000)",
    xaxis_title="Years since 1990",
    yaxis_title="CO₂ Concentration (ppm)",
    height=400
)

### Your Task

Build a simple GP model using **either `pm.gp.Marginal` or `pm.gp.Latent`** to fit the CO₂ data. You'll need to:

1. Load and prepare the data (code provided below)
2. Choose an appropriate kernel (think about what patterns you're trying to capture)
3. Define priors for the kernel hyperparameters
4. Fit the model and generate predictions
5. Visualize the fit

Don't worry if your fit doesn't capture all the patterns perfectly—that's intentional! In the next section, we'll see how to improve this model dramatically using additive kernels.

**Prompt suggestion**: "Help me implement a GP regression model in PyMC to fit atmospheric CO₂ data. Use either Marginal or Latent formulation with an appropriate kernel, priors for lengthscale and amplitude, and generate posterior predictions with uncertainty bands."


In [ ]:
# YOUR LLM-ASSISTED CODE HERE
# Build your GP model below. Consider:
# - Which kernel captures smooth variation?
# - What are reasonable lengthscale values for data spanning 10 years?
# - Should you use Marginal (Gaussian likelihood) or Latent?
# - How will you handle the observation noise?

def fit_simple_co2_gp(X, y):
    """
    Fit a GP model to CO2 data.
    
    Parameters
    ----------
    X : array, shape (n, 1)
        Time points (years since 1990)
    y : array, shape (n,)
        CO2 concentrations (ppm)
    
    Returns
    -------
    model : pm.Model
        The fitted PyMC model
    trace : az.InferenceData
        Posterior samples
    """
    # YOUR CODE HERE
    pass

# Test your implementation
# model_co2, trace_co2 = fit_simple_co2_gp(X_co2, y_co2)
# az.summary(trace_co2, var_names=['ell', 'eta', 'sigma'])

### Reflecting on Your Simple GP Fit

Once you've fit your model, take a moment to examine the results carefully. If you used a single smooth kernel like ExpQuad or Matérn, you likely noticed something important: **the model had to make a compromise**.

**If you chose a short lengthscale**: The GP might have captured some of the seasonal oscillations, but at the cost of creating unrealistic wiggles in the long-term trend. The model is trying to fit both patterns with a single smoothness parameter, and it's struggling.

**If you chose a long lengthscale**: The GP might have captured the decade-scale upward trend beautifully, giving you a smooth increasing curve. But look closely at the residuals—you'll see a clear annual pattern that the model completely missed. The seasonal cycle is still there in the data; your model just couldn't see it.

**This is the fundamental limitation**: A single kernel with one lengthscale can only capture variation at one characteristic timescale. The CO₂ data has variation at **two very different timescales**—years (trend) and months (seasonality)—and they arise from completely independent physical processes.

Here's the key insight: these patterns don't interact; they simply add together. The long-term trend from fossil fuel emissions doesn't change the seasonal amplitude, and the seasonal cycle from vegetation doesn't affect the long-term trend. They're independent sources of variation.

This is exactly the scenario where **additive kernels** shine. In the next section, we'll see how combining kernels through addition allows us to model both patterns simultaneously, each with its own characteristic lengthscale. The improvement will be dramatic.


## Section 2.4: Additive and Multiplicative Kernels

One of the most powerful features of GPs is that we can **compose kernels** through addition and multiplication. This allows us to build sophisticated models that capture multiple patterns simultaneously—exactly what we need for the CO₂ data you just worked with.

### The Algebra of Kernels: AND vs OR

If $k_1$ and $k_2$ are valid covariance functions, then both $k_1 + k_2$ and $k_1 \times k_2$ are also valid covariance functions. But they mean very different things:

**Addition (OR operation): $k_1 + k_2$**

Think of addition as saying "the function can vary in ways explained by kernel 1 **OR** kernel 2 (or both)." The total covariance is the sum of covariances from each kernel:

$$
k_{\text{sum}}(x, x') = k_1(x, x') + k_2(x, x')
$$

Covariance is high if **either kernel** suggests that $f(x)$ and $f(x')$ should be similar. This is perfect for modeling **independent sources of variation** that simply add together without interacting.

**Multiplication (AND operation): $k_1 \times k_2$**

Think of multiplication as saying "the function must satisfy constraints from kernel 1 **AND** kernel 2 simultaneously." The total covariance is the product:

$$
k_{\text{product}}(x, x') = k_1(x, x') \times k_2(x, x')
$$

Covariance is high only when **both kernels** agree that $f(x)$ and $f(x')$ should be similar. This is perfect for modeling situations where **one pattern modulates another**—where one kernel acts as an envelope or mask for the other.

Let's make this concrete with examples, starting with the additive case.

### Intuitive Examples of Addition vs Multiplication

To build intuition, let's think through some real-world scenarios:

**Addition (Independent Effects)**

*Example 1: Daily Temperature*
Temperature has a yearly seasonal cycle (winter is cold, summer is hot) **OR** local weather variations (day-to-day fluctuations). These are independent: the yearly cycle doesn't change the day-to-day weather variance, and local weather doesn't affect the seasonal pattern. Model: $k_{\text{seasonal}} + k_{\text{local}}$

*Example 2: Website Traffic*
A website might have a weekly pattern (more traffic on weekdays) **OR** a long-term growth trend (increasing users over months/years). These effects stack independently. Model: $k_{\text{periodic}} + k_{\text{trend}}$

*Example 3: Mauna Loa CO₂*
We saw this in our earlier example: a decade-long upward trend from fossil fuel emissions **OR** annual oscillations from vegetation. Neither affects the other—they're independent physical processes. Model: $k_{\text{trend}} + k_{\text{seasonal}}$

**Multiplication (Modulation and Interaction)**

*Example 1: Seasonal Sales Patterns During Business Growth*
A retail business has weekly sales patterns (higher on weekends) **AND** those patterns only emerged as the business matured—they weren't present in year 1. The growth process modulates the strength of the weekly pattern. Model: $k_{\text{linear}} \times k_{\text{periodic}}$

*Example 2: Equipment Vibrations*
A machine might vibrate periodically (due to rotating parts) **AND** those vibrations only occur when the machine is running at certain speeds (captured by a smooth function of operating conditions). The operating regime gates whether oscillations exist. Model: $k_{\text{smooth}} \times k_{\text{periodic}}$ (locally periodic)

*Example 3: Spatial-Temporal Phenomena*
Consider pollution levels that vary both in space and time, but the spatial patterns change slowly over time. The temporal pattern modulates which spatial structures are relevant at any moment. Model: $k_{\text{spatial}}(x, x') \times k_{\text{temporal}}(t, t')$

**Mental Model Summary**

- **Addition**: "Stack the effects" — independent contributions that simply superimpose
- **Multiplication**: "Mask or modulate" — one pattern determines where/when the other pattern appears

When building models, ask yourself: "Are these patterns independent sources of variation (addition), or does one pattern control the expression of the other (multiplication)?" This will guide your kernel composition strategy.

Now let's see additive kernels in action by returning to the CO₂ data with a proper decomposition.

### Additive Kernels for the CO₂ Data

Let's return to the Mauna Loa CO₂ data from the previous exercise and see how additive kernels solve the problem you likely encountered. Remember, a single kernel had to compromise between capturing the long-term trend and the seasonal oscillations.

The solution is remarkably elegant: we'll use **two kernels added together**, each modeling a different physical process:

1. **ExpQuad kernel with long lengthscale**: Captures the smooth decade-scale trend from cumulative emissions
2. **Periodic kernel with 1-year period**: Captures the annual seasonal cycle from vegetation dynamics

When we add these kernels, $k_{\text{total}} = k_{\text{trend}} + k_{\text{seasonal}}$, the resulting GP can simultaneously model both patterns, each with appropriate characteristic scales. Let's build this model.

We'll use the same data you loaded in the previous exercise (`X_co2` and `y_co2`). Now let's build an additive model that combines a smooth trend kernel with a periodic seasonal kernel.

### Observing the Independent Patterns

Looking at the data, two distinct patterns are immediately apparent:

**The long-term trend**: CO₂ concentrations rise steadily over the decade, from ~355 ppm in 1990 to ~370 ppm in 2000. This reflects cumulative emissions from fossil fuel combustion and land use changes.

**The seasonal cycle**: Regular oscillations repeat annually with an amplitude of about 5-7 ppm. Each spring, CO₂ concentrations drop as Northern Hemisphere plants leaf out and photosynthesize. Each fall, concentrations rise as plants shed leaves and decomposition releases CO₂.

The key insight: these patterns arise from **independent physical processes**. The trend results from anthropogenic emissions. The seasonal cycle results from the terrestrial biosphere's annual growth/decay cycle. Neither process modulates the other—they simply add together.

This is exactly when we need an additive kernel: $k_{\text{total}} = k_{\text{trend}} + k_{\text{seasonal}}$. Let's build the model.

### Building an Additive GP: Trend + Seasonal Components

We'll use an **additive kernel** to model the trend and seasonality as independent components:

- **Matérn 5/2 kernel** with a long lengthscale for the decadal trend
- **Periodic kernel** with period=1 year for the seasonal oscillations

The mathematical insight: when we add kernels, the resulting GP's covariance function is

$$
k_{\text{total}}(x, x') = k_{\text{trend}}(x, x') + k_{\text{seasonal}}(x, x')
$$

This means the **total variance** is the sum of variances from each component. Each kernel can explain different aspects of the data independently. The GP will automatically learn how much variation to attribute to the trend versus the seasonal cycle.

Let's implement this model.

In [ ]:
with pm.Model() as additive_co2_model:
    
    # Trend component: smooth, long-term variation
    ell_trend = pm.Gamma('ell_trend', alpha=2, beta=0.5)  # Prior favors larger lengthscales
    eta_trend = pm.HalfNormal('eta_trend', sigma=10)  # Allow for substantial trend magnitude
    cov_trend = eta_trend**2 * pm.gp.cov.Matern52(1, ell_trend)
    
    # Seasonal component: periodic with yearly period
    ell_seasonal = pm.Gamma('ell_seasonal', alpha=2, beta=2)
    eta_seasonal = pm.HalfNormal('eta_seasonal', sigma=5)  # Seasonal amplitude ~3-7 ppm
    period = 1.0  # One year
    cov_seasonal = eta_seasonal**2 * pm.gp.cov.Periodic(1, period=period, ls=ell_seasonal)
    
    # ADDITIVE KERNEL: sum of trend and seasonal
    cov_total = cov_trend + cov_seasonal
    
    # GP with additive kernel
    gp = pm.gp.Marginal(cov_func=cov_total)
    
    # Likelihood
    sigma = pm.HalfNormal('sigma', sigma=1)  # Measurement noise
    y_obs = gp.marginal_likelihood('y', X=X_co2, y=y_co2, sigma=sigma)
    
    # Sample
    additive_co2_trace = pm.sample(
        500,
        tune=1000,
        nuts_sampler='nutpie',
        chains=2,
        random_seed=RNG
    )

### Understanding the Additive Model Structure

The critical line is `cov_total = cov_trend + cov_seasonal`. This creates a new covariance function where:

- The **trend component** can vary slowly over years, capturing the decadal increase in CO₂
- The **seasonal component** can oscillate with a one-year period, capturing the annual vegetation cycle
- The **total variation** is the sum of both—they contribute independently

We gave each component its own hyperparameters:
- `ell_trend`, `eta_trend`: Control how smooth and how large the long-term trend is
- `ell_seasonal`, `eta_seasonal`: Control how smooth each annual cycle is and how large the seasonal amplitude is

The model will learn these parameters from the data, effectively **decomposing** the CO₂ signal into its constituent physical processes.

Let's visualize the results to see how well the additive kernel captures both patterns.

In [ ]:
# Make predictions (including slight extrapolation)
X_pred_co2 = np.linspace(-0.5, 11, 400)[:, None]

with additive_co2_model:
    f_pred_co2 = gp.conditional('f_pred', X_pred_co2)
    ppc_co2 = pm.sample_posterior_predictive(
        additive_co2_trace,
        var_names=['f_pred'],
        random_seed=RNG
    )

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

pm.gp.util.plot_gp_dist(
    ax=ax,
    x=X_pred_co2.flatten(),
    samples=ppc_co2.posterior_predictive['f_pred'].values.reshape(-1, len(X_pred_co2)),
    plot_samples=False, 
    palette='Reds',   
)

ax.scatter(
    df_co2['time'],
    df_co2['CO2'],
    s=20,
    c='black',
    alpha=0.6,
    label='Observed CO₂',
    zorder=10
)

ax.set_title('Additive GP: Trend + Seasonal Components', fontsize=12)
ax.set_xlabel('Years since 1990', fontsize=10)
ax.set_ylabel('CO₂ concentration (ppm)', fontsize=10)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Interpreting the Additive Model Results

Beautiful! The additive GP has successfully decomposed the Mauna Loa CO₂ data into its constituent patterns:

**The trend component** captures the steady decade-long increase from fossil fuel emissions. The model learned that this variation happens on a time scale of years, with a large lengthscale parameter.

**The seasonal component** captures the annual oscillations from vegetation dynamics. The model learned that this pattern repeats exactly every year (period=1.0) with an amplitude of several ppm.

**The combined fit** (dark red line) shows how these independent patterns add together to explain the observations. The credible intervals (shown as progressively lighter shades of red for 68%, 95%, and wider intervals) reflect our uncertainty—notice how the uncertainty bands grow slightly when extrapolating beyond year 10.

This is the power of additive models: when your data contains multiple **independent** sources of variation, each kernel explains a different aspect. The GP automatically learns how to decompose the signal based on the different correlation structures (long-term smooth trend vs. yearly periodic cycle).

**Key intuition**: Addition works when patterns don't interact—when one doesn't modulate the other. The CO₂ trend doesn't change the seasonal amplitude, and the seasonal cycle doesn't affect the long-term trend. They're independent physical processes that simply superimpose.

Now let's see what happens when patterns **do** interact—that's where multiplicative kernels come in.

### Locally Periodic Patterns: When Seasonality Isn't Global

The additive model we just built assumed that both the trend and seasonal patterns exist throughout the entire time series. But what if your periodic pattern only appears in certain regions, or gradually fades in and out?

Consider these scenarios:
- **Ecological data**: Animal behavior might be seasonal in temperate regions but constant in tropical regions
- **Economic data**: Holiday sales patterns might emerge over time as a business matures
- **Sensor data**: Periodic noise might only appear when equipment is under certain operating conditions

For these cases, we need a **locally periodic kernel**—a periodic pattern that's modulated by a smooth envelope. The key insight: this requires **multiplication**, not addition.

Recall the algebra of kernel composition:
- **Addition** (k₁ + k₂): Independent sources of variation—"this OR that"
- **Multiplication** (k₁ × k₂): One pattern modulates another—"this AND that"

A **locally periodic kernel** multiplies a Periodic kernel with a smooth kernel (typically ExpQuad or Matérn):

$$
k_{\text{local-periodic}}(x, x') = k_{\text{smooth}}(x, x') \times k_{\text{periodic}}(x, x')
$$

This creates periodic oscillations that are strong where the smooth kernel has high covariance, and weak (or absent) where it doesn't. Let's see this in action with synthetic data.

In [ ]:
# Generate data with localized periodic pattern
np.random.seed(RANDOM_SEED)

X_local_per = np.linspace(0, 15, 250)[:, None]

# Create a Gaussian envelope centered at x=7.5
center = 7.5
envelope_width = 3.0
envelope = np.exp(-0.5 * ((X_local_per.flatten() - center) / envelope_width)**2)

# Periodic component with annual period
periodic_component = np.sin(2 * np.pi * X_local_per.flatten() / 2.0)

f_true_local = 2.0 * envelope * periodic_component

y_local_per = f_true_local + 0.3 * RNG.standard_normal(len(X_local_per))

df_local_per = pl.DataFrame({
    'x': X_local_per.flatten(),
    'y': y_local_per,
    'f_true': f_true_local,
    'envelope': 2.0 * envelope,
    'periodic': 2.0 * periodic_component
})

go.Figure().add_trace(go.Scatter(
    x=df_local_per['x'],
    y=df_local_per['f_true'],
    mode='lines',
    name='True function (localized periodic)',
    line=dict(color='dodgerblue', width=2)
)).add_trace(go.Scatter(
    x=df_local_per['x'],
    y=df_local_per['envelope'],
    mode='lines',
    name='Envelope (Gaussian)',
    line=dict(color='green', width=2, dash='dash')
)).add_trace(go.Scatter(
    x=df_local_per['x'],
    y=df_local_per['y'],
    mode='markers',
    name='Observed data',
    marker=dict(size=4, color='black', opacity=0.6)
)).update_layout(
    title='Data with Localized Periodic Pattern',
    xaxis_title='X',
    yaxis_title='Y',
    hovermode='x unified',
    width=900,
    height=500
)

We'll fit three models to compare approaches:

1. Pure Periodic (wrong - assumes global periodicity)
2. ExpQuad + Periodic (wrong - independent components)
3. ExpQuad × Periodic (correct - modulated periodicity)

In [ ]:
# Model 1: Pure Periodic
with pm.Model() as model_pure_periodic:
    ell_per = pm.Gamma('ell_per', alpha=2, beta=2)
    eta_per = pm.HalfNormal('eta_per', sigma=3)
    cov_per = eta_per**2 * pm.gp.cov.Periodic(1, period=2.0, ls=ell_per)

    gp = pm.gp.Marginal(cov_func=cov_per)
    sigma = pm.HalfNormal('sigma', sigma=1)
    y_obs = gp.marginal_likelihood('y', X=X_local_per, y=y_local_per, sigma=sigma)

    trace_pure_per = pm.sample(500, tune=500, nuts_sampler='nutpie', chains=2, random_seed=RNG)

X_pred_local = np.linspace(-1, 16, 400)[:, None]

with model_pure_periodic:
    
    f_pred_pure = gp.conditional('f_pred', X_pred_local)
    ppc_pure = pm.sample_posterior_predictive(trace_pure_per, var_names=['f_pred'], random_seed=RNG)


In [ ]:
# Model 2: Additive (ExpQuad + Periodic)
with pm.Model() as model_additive:
    # Smooth component
    ell_smooth = pm.Gamma('ell_smooth', alpha=2, beta=0.5)
    eta_smooth = pm.HalfNormal('eta_smooth', sigma=3)
    cov_smooth = eta_smooth**2 * pm.gp.cov.ExpQuad(1, ell_smooth)

    # Periodic component
    ell_per = pm.Gamma('ell_per', alpha=2, beta=2)
    eta_per = pm.HalfNormal('eta_per', sigma=3)
    cov_per = eta_per**2 * pm.gp.cov.Periodic(1, period=2.0, ls=ell_per)

    # Addition
    cov_add = cov_smooth + cov_per

    gp = pm.gp.Marginal(cov_func=cov_add)
    sigma = pm.HalfNormal('sigma', sigma=1)
    y_obs = gp.marginal_likelihood('y', X=X_local_per, y=y_local_per, sigma=sigma)

    trace_add = pm.sample(500, tune=500, nuts_sampler='nutpie', chains=2, random_seed=RNG)

with model_additive:
    
    f_pred_add = gp.conditional('f_pred', X_pred_local)
    ppc_add = pm.sample_posterior_predictive(trace_add, var_names=['f_pred'], random_seed=RNG)


In [ ]:
# Model 3: Multiplicative (ExpQuad × Periodic) - LOCALLY PERIODIC
with pm.Model() as model_locally_periodic:
    # Smooth component
    ell_smooth = pm.Gamma('ell_smooth', alpha=2, beta=0.5)
    eta_smooth = pm.HalfNormal('eta_smooth', sigma=3)
    cov_smooth = eta_smooth**2 * pm.gp.cov.ExpQuad(1, ell_smooth)

    # Periodic component
    ell_per = pm.Gamma('ell_per', alpha=2, beta=2)
    eta_per = pm.HalfNormal('eta_per', sigma=3)
    cov_per = eta_per**2 * pm.gp.cov.Periodic(1, period=2.0, ls=ell_per)

    # Multiplication - this creates local periodicity
    cov_mult = cov_smooth * cov_per

    gp = pm.gp.Marginal(cov_func=cov_mult)
    sigma = pm.HalfNormal('sigma', sigma=1)
    y_obs = gp.marginal_likelihood('y', X=X_local_per, y=y_local_per, sigma=sigma)

    trace_mult = pm.sample(500, tune=500, nuts_sampler='nutpie', chains=2, random_seed=RNG)

with model_locally_periodic:
    
    f_pred_mult = gp.conditional('f_pred', X_pred_local)
    ppc_mult = pm.sample_posterior_predictive(trace_mult, var_names=['f_pred'], random_seed=RNG)


Let's visualize all three fits side-by-side:


In [ ]:
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Pure Periodic', 'Additive', 'Locally Periodic'),
    horizontal_spacing=0.1,
    shared_yaxes=True
)

def extract_predictions(ppc):
    samples = ppc.posterior_predictive['f_pred'].values
    mean = samples.mean(axis=(0, 1))
    lower = np.percentile(samples, 2.5, axis=(0, 1))
    upper = np.percentile(samples, 97.5, axis=(0, 1))
    return mean, lower, upper

for idx, (ppc, title) in enumerate([(ppc_pure, 'Pure'), (ppc_add, 'Additive'), (ppc_mult, 'Multiplicative')], start=1):
    mean, lower, upper = extract_predictions(ppc)

    fig.add_trace(go.Scatter(x=X_pred_local.flatten(), y=upper, mode='lines',
                            line=dict(width=0), showlegend=False, hoverinfo='skip'),
                 row=1, col=idx)
    fig.add_trace(go.Scatter(x=X_pred_local.flatten(), y=lower, mode='lines',
                            fill='tonexty', fillcolor='rgba(255, 0, 0, 0.2)',
                            line=dict(width=0), showlegend=False, hoverinfo='skip'),
                 row=1, col=idx)

    fig.add_trace(go.Scatter(x=X_pred_local.flatten(), y=mean, mode='lines',
                            name='Posterior mean', line=dict(color='red', width=2),
                            showlegend=(idx==1)),
                 row=1, col=idx)

    fig.add_trace(go.Scatter(x=df_local_per['x'], y=df_local_per['f_true'],
                            mode='lines', name='True function',
                            line=dict(color='dodgerblue', width=2, dash='dash'),
                            showlegend=(idx==1)),
                 row=1, col=idx)

    fig.add_trace(go.Scatter(x=df_local_per['x'], y=df_local_per['y'],
                            mode='markers', name='Data',
                            marker=dict(size=3, color='black', opacity=0.5),
                            showlegend=(idx==1)),
                 row=1, col=idx)

fig.update_yaxes(range=[-5, 5])
fig.update_layout(height=400, width=1200)

### Why Multiplication Worked Where Addition Failed

Looking at the three models side by side reveals the critical difference:

**Model 1: Pure Periodic (Left Panel)**
This model assumes the periodic pattern exists everywhere with constant amplitude. Outside the central region where the true periodicity exists, the model confidently predicts oscillations that aren't there. This is exactly wrong—it extrapolates periodic behavior globally when the data only shows it locally.

**Model 2: Additive (Center Panel)**
The additive model tried to explain the data with two independent components: a smooth function plus a global periodic function. While it might capture some structure, it fundamentally misunderstands the data generation process. The smooth component can't "turn off" the periodic component in regions where it shouldn't exist. This model will likely predict some oscillations outside the central region (though less extreme than Model 1) and may underestimate the periodic amplitude in the central region where it's strong.

**Model 3: Locally Periodic (Right Panel)**
The multiplicative model correctly captures the localized periodicity. The posterior mean (red line) closely follows the true function (blue dashed line), showing strong oscillations in the central region and smooth, non-oscillatory behavior outside. The credible interval grows appropriately in regions far from the data, but it doesn't confidently predict spurious oscillations.

**The mathematical insight**: When you multiply kernels, the covariance $k_1(x, x') \times k_2(x, x')$ is high only when **both** kernels have high covariance. The ExpQuad kernel creates an envelope of high covariance around the central region. The Periodic kernel creates high covariance at regular intervals. Their product creates periodic covariance **only within the envelope**—exactly what we wanted.

**When to use locally periodic kernels**:
- Seasonal patterns that emerge or fade over time
- Periodic phenomena that only occur in specific regions of input space
- Oscillatory behavior that's modulated by another process

**Key takeaway**: Multiplication is not just for "growing amplitude"—it's the right tool whenever one pattern should **gate** or **localize** another. The order doesn't matter in multiplication (k₁ × k₂ = k₂ × k₁), but the interpretation does: think of one kernel as the "envelope" and the other as the "carrier signal."

Now let's continue exploring multiplicative patterns with another example.

## 🤖 EXERCISE: Multiplicative Kernels with Air Passengers Data

You've seen how additive kernels work for the CO₂ data, where trend and seasonality are independent. Now let's explore **multiplicative seasonality**, where seasonal amplitude grows over time.

### Understanding Multiplicative Seasonality

The Air Passengers dataset shows monthly totals from 1949-1960. Notice how seasonal oscillations grow larger over time—early years have small swings (±10-20), late years have large swings (±100-200). The seasonal amplitude is **proportional to the trend level**.

This is different from the CO₂ data, where seasonality has constant amplitude. Here, one pattern (the trend) **modulates** another pattern (the seasonality).

In [ ]:
# Load Air Passengers data from PyMC
df_air = pl.read_csv(pm.get_data('AirPassengers.csv'))

# Create time index in years
df_air = df_air.with_columns([
    pl.Series('time_years', [i/12.0 for i in range(len(df_air))])
])

X_air = df_air['time_years'].to_numpy()[:, None]
y_air = df_air['#Passengers'].to_numpy().astype(float)

print(f"Data: {len(X_air)} monthly observations from 1949-1960")
print(f"Passenger range: {y_air.min():.0f} to {y_air.max():.0f}")

# Visualize
go.Figure().add_trace(go.Scatter(
    x=X_air.flatten(), y=y_air,
    mode='markers+lines',
    marker=dict(size=3, color='darkblue'),
    line=dict(width=1, color='steelblue')
)).update_layout(
    title='Air Passengers (1949-1960)',
    xaxis_title='Years since 1949',
    yaxis_title='Passengers',
    height=400
)

### Your Task: Modeling Growing Seasonality

Build a GP model using **multiplicative kernels** to capture the growing seasonal amplitude in the Air Passengers data.

Notice how the seasonal fluctuations grow larger over time—the swings in 1949 are much smaller than in 1960. This requires **multiplication** of kernels, not addition.

**Think about**:
- Which kernels capture trend and seasonality? (ExpQuad × Periodic)
- What lengthscales make sense for 12 years of data?
- What is the period for yearly cycles?

**Success**: Your model should show small seasonal swings early and large swings later.



In [ ]:
# YOUR LLM-ASSISTED CODE HERE
# Build your GP model below. Consider:
# - Which kernel captures trend and seasonality?
# - What are reasonable lengthscale values for data spanning 12 years?
# - How will you handle the observation noise?

# Test your implementation


# ====================
# PLOTTING CODE
# ====================
# Uncomment after building your model (assumes: model, gp defined)

# X_pred = np.linspace(-1, 13, 300)[:, None]
# 
# with model:
#     f_pred = gp.conditional('f_pred', X_pred)
# 
# mu = f_pred.eval()
# 
# # Plot with credible intervals
# fig = go.Figure()
# 
# for n_std, alpha in [(3, 0.1), (2, 0.15), (1, 0.25)]:
#     sigma = np.sqrt(gp.conditional('f_pred', X_pred).eval())
#     fig.add_trace(go.Scatter(
#         x=np.concatenate([X_pred.flatten(), X_pred.flatten()[::-1]]),
#         y=np.concatenate([mu + n_std*sigma, (mu - n_std*sigma)[::-1]]),
#         fill='toself', fillcolor=f'rgba(255,100,100,{alpha})',
#         line=dict(width=0), showlegend=False, hoverinfo='skip'
#     ))
# 
# fig.add_trace(go.Scatter(x=X_pred.flatten(), y=mu, 
#                          line=dict(color='darkred', width=2), name='GP Mean'))
# fig.add_trace(go.Scatter(x=X_air.flatten(), y=y_air, mode='markers',
#                          marker=dict(color='darkblue', size=4), name='Data'))
# 
# fig.update_layout(title='Multiplicative GP: Growing Seasonal Amplitude',
#                   xaxis_title='Years since 1949', yaxis_title='Passengers',
#                   height=450)
# fig.show()


### Encoding Domain Knowledge Through Kernel Design

Beyond choosing from standard kernels, you can encode specific domain knowledge directly into kernel structure. Here are a few advanced patterns:

**Symmetry**

If you know your function has reflective symmetry around the origin ($f(x) = f(-x)$), you can enforce this:

$$
k_{\text{symmetric}}(x, x') = k(x, x') + k(-x, x')
$$

This doubles the effective data by using the symmetry constraint. For example, if you're modeling physical phenomena that must be symmetric (like the electric field around a charged sphere), enforcing this constraint improves efficiency and prevents the model from learning spurious asymmetries.

**Low-Dimensional Structure**

If you suspect your high-dimensional input actually varies along a low-dimensional manifold, project before applying the kernel:

$$
k_{\text{low-dim}}(\mathbf{x}, \mathbf{x}') = k(\mathbf{A}\mathbf{x}, \mathbf{A}\mathbf{x}')
$$

where $\mathbf{A}$ is a low-rank matrix (e.g., $m \times d$ where $m \ll d$). This is like doing dimensionality reduction before GP regression. You can learn $\mathbf{A}$ jointly with the GP, creating a model that discovers the relevant low-dimensional structure.

**Categorical Variables**

For categorical inputs (e.g., "red", "blue", "green"), use one-hot encoding with product kernels:

```python
# For 2D input: continuous variable and 3-level categorical
# Encode categorical as [1,0,0], [0,1,0], [0,0,1]
with pm.Model() as model:
    # Continuous dimension
    ell_cont = pm.Gamma('ell_cont', alpha=2, beta=1)
    cov_continuous = pm.gp.cov.Matern52(1, ls=ell_cont)
    
    # Categorical dimension (3 levels encoded as 3D one-hot)
    ell_cat = pm.Gamma('ell_cat', alpha=2, beta=1, shape=3)
    cov_categorical = pm.gp.cov.ExpQuad(3, ls=ell_cat)
    
    # Combine multiplicatively
    cov_total = cov_continuous * cov_categorical
```

The model learns lengthscales for the categorical encoding, effectively learning whether categories should share information. Small lengthscales mean the categories are very different; large lengthscales mean they're similar.

**Periodic Boundary Conditions**

For phenomena defined on a circle (e.g., compass directions, time of day), the input space wraps around. Instead of treating 359° and 1° as distant, encode the input as $(\cos(\theta), \sin(\theta))$ and use a standard kernel on this 2D representation.

**When to use these patterns**: Only when you have strong domain knowledge that justifies the constraint. Imposing wrong symmetry or structure will harm performance. When in doubt, stick with simpler kernels and let the data speak.

These advanced patterns are mostly relevant for specialized applications. For most practical GP modeling, you'll succeed with the standard kernels and composition rules we've explored. The key is to start simple and add complexity only when data or domain knowledge clearly indicates it's needed.

Now that we have a comprehensive understanding of kernels and their properties, let's see how to handle non-Gaussian data through flexible likelihood choices.

## Section 2.5: Non-Gaussian Likelihoods - Robust Regression with Student-T

So far, we've used Gaussian noise (in `pm.gp.Marginal`) or Bernoulli outcomes (in classification). But real data often contains **outliers**—observations that don't fit the typical pattern. Gaussian distributions are not robust to outliers: a single extreme value can dramatically affect inference.

The **Student-T distribution** offers a robust alternative. It has heavier tails than a Gaussian, meaning it assigns higher probability to extreme values. This makes it much more tolerant of outliers.

### Why Student-T for Robust Regression?

The Student-T distribution has three parameters:
- **Location ($\mu$)**: The center, similar to the mean
- **Scale ($\sigma$)**: The spread, similar to standard deviation
- **Degrees of freedom ($\nu$)**: Controls tail heaviness

The degrees of freedom parameter is key:
- Small $\nu$ (e.g., 2-5) → very heavy tails, very robust to outliers
- Large $\nu$ (> 30) → approaches normal distribution

Since we need `pm.gp.Latent` for non-Gaussian likelihoods, let's see robust regression in action with Student-T likelihood.

### Example: Data with Outliers

Let's generate data where most observations are clean, but a few are extreme outliers.

In [ ]:
# Generate data with outliers
n_robust = 100
X_robust = np.linspace(0, 10, n_robust)[:, None]

# True function
f_true_robust = 2 * np.sin(1.5 * X_robust.flatten())

# Most observations have small Gaussian noise
y_robust = f_true_robust + 0.3 * RNG.standard_normal(n_robust)

# Add outliers: replace 10% of points with extreme values
n_outliers = 10
outlier_indices = RNG.choice(n_robust, size=n_outliers, replace=False)
y_robust[outlier_indices] += RNG.choice([-1, 1], size=n_outliers) * RNG.uniform(3, 6, size=n_outliers)

df_robust = pl.DataFrame({
    'x': X_robust.flatten(),
    'y': y_robust,
    'f_true': f_true_robust,
    'is_outlier': [i in outlier_indices for i in range(n_robust)]
})

# Visualize
fig = go.Figure()

# Regular points
df_regular = df_robust.filter(pl.col('is_outlier') == False)
fig.add_trace(go.Scatter(
    x=df_regular['x'],
    y=df_regular['y'],
    mode='markers',
    name='Regular observations',
    marker=dict(size=5, color='black')
))

# Outliers
df_outlier = df_robust.filter(pl.col('is_outlier') == True)
fig.add_trace(go.Scatter(
    x=df_outlier['x'],
    y=df_outlier['y'],
    mode='markers',
    name='Outliers',
    marker=dict(size=8, color='red', symbol='x')
))

# True function
fig.add_trace(go.Scatter(
    x=df_robust['x'],
    y=df_robust['f_true'],
    mode='lines',
    name='True function',
    line=dict(color='dodgerblue', width=3)
))

fig.update_layout(
    title='Data with Outliers',
    xaxis_title='X',
    yaxis_title='Y',
    hovermode='x unified',
    width=900,
    height=500
)

fig.show()

### Observing the Outliers

The red X markers show outliers—observations that are far from the true function (blue line). If we used a Gaussian likelihood, these outliers would pull our posterior predictions toward them, distorting our estimate of the true function.

Let's build a robust model using a Student-T likelihood with `pm.gp.Latent`.

In [ ]:
with pm.Model() as robust_model:
    # Hyperparameters
    ell = pm.Gamma('ell', alpha=2, beta=1)
    eta = pm.HalfNormal('eta', sigma=3)
    
    # Covariance function
    cov_func = eta**2 * pm.gp.cov.Matern52(1, ell)
    
    # Latent GP
    gp = pm.gp.Latent(cov_func=cov_func)
    f = gp.prior('f', X=X_robust)
    
    # Student-T likelihood for robustness
    sigma = pm.HalfNormal('sigma', sigma=1)
    nu = pm.Gamma('nu', alpha=2, beta=0.1)  # Prior favors low nu (heavy tails)
    
    y_obs = pm.StudentT(
        'y',
        mu=f,
        sigma=sigma,
        nu=nu,
        observed=y_robust
    )
    
    # Sample
    robust_trace = pm.sample(
        500,
        tune=500,
        nuts_sampler='nutpie',
        chains=2,
        target_accept=0.9,
        random_seed=RNG
    )

### Understanding the Robust Model

The key difference from our earlier Gaussian models is the likelihood:

```python
y_obs = pm.StudentT('y', mu=f, sigma=sigma, nu=nu, observed=y_robust)
```

We're learning the degrees of freedom parameter `nu` from the data. The prior `Gamma(2, 0.1)` suggests we expect heavy tails (small nu), but leaves room for the data to tell us otherwise.

Let's see how well this handles the outliers.

In [ ]:
# Check the learned nu parameter
nu_summary = az.summary(robust_trace, var_names=['nu'])
print("Degrees of freedom posterior:")
print(nu_summary)
print(f"\nMean nu: {nu_summary['mean'].values[0]:.2f}")
print("(Small nu indicates heavy tails were needed)")

### Making Robust Predictions

Now let's predict and see if the model successfully ignored the outliers.

In [ ]:
# Predictions
X_pred_robust = np.linspace(-1, 11, 300)[:, None]

USE_CACHED = True

if USE_CACHED and os.path.exists(OUTPUT_DIR + "robust_ppc.nc"):
    ppc_robust = az.from_netcdf(OUTPUT_DIR + "robust_ppc.nc")

else:

    with robust_model:
        f_pred_robust = gp.conditional('f_pred', X_pred_robust)
        ppc_robust = pm.sample_posterior_predictive(
            robust_trace,
            var_names=['f_pred'],
            random_seed=RNG
        )
    ppc_robust.to_netcdf(OUTPUT_DIR + "robust_ppc.nc")

In [ ]:
# Visualize using PyMC's built-in plotting function
fig, ax = plt.subplots(figsize=(10, 5))

# Plot the GP posterior distribution
pm.gp.util.plot_gp_dist(
    ax=ax,
    x=X_pred_robust.flatten(),
    samples=ppc_robust.posterior_predictive['f_pred'].values.reshape(-1, len(X_pred_robust)),
    plot_samples=False, 
    palette='Reds'
)

# True function
ax.plot(
    df_robust['x'],
    df_robust['f_true'],
    color='dodgerblue',
    linewidth=2,
    linestyle='--',
    label='True function'
)

# Regular observations
ax.scatter(
    df_regular['x'],
    df_regular['y'],
    s=25,
    color='black',
    label='Regular observations',
    zorder=5
)

# Outliers
ax.scatter(
    df_outlier['x'],
    df_outlier['y'],
    s=64,
    color='red',
    marker='x',
    linewidths=2,
    label='Outliers',
    zorder=5
)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('Robust GP with Student-T Likelihood')
ax.legend()
plt.tight_layout()
plt.show()


### Interpreting the Robust Results

Excellent! The posterior mean (red line) closely follows the true function (blue dashed line), essentially ignoring the outliers (red X markers). The Student-T likelihood has successfully downweighted the extreme observations.

Compare this to what would happen with a Gaussian likelihood—the outliers would pull the predicted function toward them, creating bulges in the estimate. The Student-T likelihood's heavy tails allow the model to say "these observations are just noise from the tail of the distribution," rather than trying to fit them.

**When to use robust likelihoods**:
- When you suspect your data contains outliers
- When measurement errors might occasionally be much larger than usual
- When you want your model to be less sensitive to a few extreme values

The cost is slower sampling (since we use `pm.gp.Latent`), but the benefit is much more reliable inference in the presence of outliers.